# Control de la dirección

Hasta ahora, hemos tratado el robot como un sistema plano, considerando que se mueve siempre en línea recta. Para ello, hemos condensado los dos motores en uno solo, haciendo lo mismo con las ruedas. Esto presenta varios problemas, siendo el más evidente que el robot no puede aprovechar que tiene dos motores independientes para moverse en cualquier dirección. Incluso aunque el objetivo fuese moverse en línea recta, será difícil conseguirlo: si se utiliza un voltaje común para los dos motores, cualquier pequeña diferencia entre ellos (rozamiento, resistencia del bobinado, etc.) hará que giren a velocidades ligeramente diferentes, lo que provocará desviaciones en la trayectoria.

La solución más simple, que nos evita recurrir a un modelo completo en 3D, es introducir una segunda capa de control para la velocidad de giro alrededor del eje vertical. Necesitaremos entonces dos cosas: poder medir dicha velocidad, y saber cómo modificarla sin que se caiga el robot.

## Estimación de la velocidad de giro

Si queremos medir la velocidad de giro $\dot\phi$, tendremos que expresarla en función de variables que podamos medir. Lo más evidente es ponerla en función de las velocidades de los centros de las ruedas:

$$
    \dot\phi = \frac{\dot x_r - \dot x_l}{d}
$$

donde $\dot x_r$ y $\dot x_l$ son las velocidades de las ruedas derecha e izquierda, respectivamente, y $d$ es la distancia entre centros. Las velocidades de las ruedas, a su vez, se obtienen a partir de las velocidades de giro de los motores $\omega_r$ y $\omega_l$ de la manera siguiente:

$$
    \dot x_r = \left(\omega_r + \dot\theta\right)r \qquad \dot x_l = \left(\omega_l + \dot\theta\right)r
$$

Combinando ambas expresiones, se puede escribir la velocidad de giro como:

$$
    \dot\phi = \frac{r}{d}\left(\omega_r - \omega_l\right)
$$

Esta expresión sólo depende de las velocidades de giro de los motores, que se miden directamente en los *encoders*.

## Actuación sobre la velocidad de giro

Para modificar $\dot\phi$ habrá que alterar las velocidades de los motores. Con esto hay que tener cuidado, ya que se podría desequilibrar el robot, así que habrá que asegurar que el par resultante sobre el chasis no se vea modificado. El par total sobre el chasis será la suma de los pares de cada uno de los motores:

$$
    \tau_t = \tau_r + \tau_l
$$

Nuestro controlador LQR en 2D nos proporciona un par común para los dos motores, que es el que permite que el robot se mueva a la velocidad adecuada sin caerse. Si alteramos los pares de los motores, pero con cuidado de asegurar que su resultante no cambie, podremos hacer que el robot gire sin perder el equilibrio. Para ello, podemos definir un par de corrección $\Delta\tau$ tal que:

$$
    \tau_r = \tau_t + \Delta\tau \qquad \tau_l = \tau_t - \Delta\tau
$$

En teoría, para aplicar este $\Delta\tau$ habría que calcular a qué incremento de voltaje corresponde para cada motor, para su estado actual de velocidad y aceleración. En la práctica, modificando directamente el voltaje no se introduce un error tan grande como para que el LQR no pueda compensarlo, así que lo más sencillo es simplemente actuar sobre los voltajes de los motores de la siguiente manera:

$$
    V_r = V + \Delta V \qquad V_l = V - \Delta V
$$

donde $V$ es el voltaje común obtenido en el LQR, y $\Delta V$ es un ajuste de voltaje que vendrá determinado por el controlador de giro.

## Diseño del controlador

Como el estado que queremos controlar en este caso es estable, no es necesario recurrir a métodos avanzados, basta con utilizar un simple PI. Su entrada será el error en $\dot\phi$, y devolverá como salida una corrección de voltaje $\Delta V$, resultado de aplicar las ganancias correspondientes $K_p$ y $K_i$ al error y su integral. El error en la velocidad de giro se define como la diferencia entre la velocidad de giro deseada, $\dot\phi_d$, y la que tenemos realmente, $\dot\phi$, así que el controlador se puede resumir de la siguiente manera:

$$
\begin{align}
    \varepsilon &= \dot\phi_d - \dot\phi \\
    \Delta V &= K_p \varepsilon + K_i\int \varepsilon \mathrm dt
\end{align}
$$

La aplicación práctica es muy simple: se mide la velocidad de giro a partir de los *encoders*, se calcula el error respecto a la velocidad deseada, y se va acumulando su integral. Luego, se calcula el $\Delta V$ usando las ganancias del PI y se aplica al voltaje medio obtenido por el LQR, sumándolo al motor derecho y restándolo al izquierdo.